In [1]:
import heapq
import numpy as np

class PuzzleState:
    def __init__(self, board, parent=None, move=0, cost=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.cost = cost
        self.emptyTile = self.findEmptyTile()

    def findEmptyTile(self):
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    return (i, j)
        return None

    def is_goal(self):
        return np.array_equal(self.board, [[1, 2, 3], [8, 0, 4], [7, 6, 5]])

    def possibleMoves(self):
        moves = []
        x, y = self.emptyTile
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_board = np.copy(self.board)
                new_board[x][y], new_board[nx][ny] = new_board[nx][ny], new_board[x][y]
                moves.append(PuzzleState(new_board, self, move=(dx, dy), cost=self.cost + 1))
        return moves

    def __eq__(self, other):
        return np.array_equal(self.board, other.board)

    def __lt__(self, other):
        return self.cost < other.cost

    def __hash__(self):
        return hash(tuple(map(tuple, self.board)))

# Heuristic 1: Number of misplaced tiles
def h1(state):
    goal = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
    return sum([1 if state.board[i][j] != 0 and state.board[i][j] != goal[i][j] else 0
                for i in range(3) for j in range(3)])

# Heuristic 2: Manhattan distance
def h2(state):
    goal_positions = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        8: (1, 0), 0: (1, 1), 4: (1, 2),
        7: (2, 0), 6: (2, 1), 5: (2, 2)
    }

    dist = 0
    for i in range(3):
        for j in range(3):
            if state.board[i][j] != 0:
                x, y = goal_positions[state.board[i][j]]
                dist += abs(x - i) + abs(y - j)
    return dist


def a_star(initial_board, heuristic):
    open_list = []
    heapq.heappush(open_list, (0, PuzzleState(np.array(initial_board))))

    closed_set = set()

    while open_list:
        _, current_state = heapq.heappop(open_list)

        if current_state.is_goal():
            return current_state

        closed_set.add(current_state)

        for neighbor in current_state.possibleMoves():
            if neighbor in closed_set:
                continue

            g = neighbor.cost
            h = heuristic(neighbor)

            heapq.heappush(open_list, (g + h, neighbor))

    return None

def print_solution(state):
    solution = []
    while state is not None:
        solution.append(state.board)
        state = state.parent
    solution.reverse()

    for step, board in enumerate(solution):
        print(f"Step {step}:")
        print(board, "\n")


if __name__ == "__main__":
    initial_board = [[2, 8, 3],
                     [1, 6, 4],
                     [7, 0, 5]]

    print("Solving using Heuristic H1 (Misplaced Tiles)...")
    solution_h1 = a_star(initial_board, h1)
    print_solution(solution_h1)

    print("Solving using Heuristic H2 (Manhattan Distance)...")
    solution_h2 = a_star(initial_board, h2)
    print_solution(solution_h2)

Solving using Heuristic H1 (Misplaced Tiles)...
Step 0:
[[2 8 3]
 [1 6 4]
 [7 0 5]] 

Step 1:
[[2 8 3]
 [1 0 4]
 [7 6 5]] 

Step 2:
[[2 0 3]
 [1 8 4]
 [7 6 5]] 

Step 3:
[[0 2 3]
 [1 8 4]
 [7 6 5]] 

Step 4:
[[1 2 3]
 [0 8 4]
 [7 6 5]] 

Step 5:
[[1 2 3]
 [8 0 4]
 [7 6 5]] 

Solving using Heuristic H2 (Manhattan Distance)...
Step 0:
[[2 8 3]
 [1 6 4]
 [7 0 5]] 

Step 1:
[[2 8 3]
 [1 0 4]
 [7 6 5]] 

Step 2:
[[2 0 3]
 [1 8 4]
 [7 6 5]] 

Step 3:
[[0 2 3]
 [1 8 4]
 [7 6 5]] 

Step 4:
[[1 2 3]
 [0 8 4]
 [7 6 5]] 

Step 5:
[[1 2 3]
 [8 0 4]
 [7 6 5]] 

